## 📘 1. 데이터 불러오기

In [34]:
import pandas as pd

# 파일 결로 지정
file_path = "../data/raw/seoul_food_map.csv"

# CSV 파일 불러오기
with open(file_path, encoding="cp949", errors="replace") as f:
    df = pd.read_csv(f)

/tmp/ipykernel_52192/1164755509.py:8: DtypeWarning: Columns (36,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [35]:
df.head()

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3000000,3000000-101-1997-07612,1997-09-25,NaN,3,폐업,2,폐업,2003-06-18,NaN,...,NaN,NaN,NaN,NaN,NaN,N,10.50,NaN,NaN,NaN
1,3000000,3000000-101-1997-07610,1997-09-25,NaN,3,폐업,2,폐업,1999-02-26,NaN,...,NaN,NaN,NaN,NaN,NaN,N,40.60,NaN,NaN,NaN
2,3000000,3000000-101-2000-11659,2000-11-28,NaN,3,폐업,2,폐업,2004-03-09,NaN,...,NaN,NaN,NaN,NaN,NaN,N,23.40,NaN,NaN,NaN
3,3000000,3000000-101-2000-11660,2000-11-28,NaN,3,폐업,2,폐업,2004-05-27,NaN,...,NaN,NaN,NaN,NaN,NaN,N,15.97,NaN,NaN,NaN
4,3000000,3000000-101-1981-00796,1981-08-26,NaN,3,폐업,2,폐업,2003-05-09,NaN,...,NaN,NaN,NaN,NaN,NaN,N,211.44,NaN,NaN,NaN


In [36]:
df.columns.to_list()

['개방자치단체코드',
 '관리번호',
 '인허가일자',
 '인허가취소일자',
 '영업상태코드',
 '영업상태명',
 '상세영업상태코드',
 '상세영업상태명',
 '폐업일자',
 '휴업시작일자',
 '휴업종료일자',
 '재개업일자',
 '전화번호',
 '소재지면적',
 '소재지우편번호',
 '지번주소',
 '도로명주소',
 '도로명우편번호',
 '사업장명',
 '최종수정일자',
 '데이터갱신구분',
 '데이터갱신일자',
 '업태구분명',
 '좌표정보(X)',
 '좌표정보(Y)',
 '위생업태명',
 '남성종사자수',
 '여성종사자수',
 '영업장주변구분명',
 '등급구분명',
 '급수시설구분명',
 '총인원',
 '본사종업원수',
 '공장사무직종업원수',
 '공장판매직종업원수',
 '공장생산직종업원수',
 '건물소유구분명',
 '보증액',
 '월세액',
 '다중이용업소여부',
 '시설총규모',
 '전통업소지정번호',
 '전통업소주된음식',
 '홈페이지']

## 📘 2. 좌표 변환 (EPSG:2097 → EPSG:4326)

In [37]:
from pyproj import Transformer

# 좌표 변환기 초기화 (중부원점 -> 위경도)
transformer = Transformer.from_crs("EPSG:2097", "EPSG:4326", always_xy=True)


# 변환 함수 정의
def convert_coords(x, y):
    try:
        lon, lat = transformer.transform(x, y)
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

In [38]:
# 좌표 변환 실행
df[["위도", "경도"]] = df.apply(
    lambda row: convert_coords(row["좌표정보(X)"], row["좌표정보(Y)"]), axis=1
)

# 결과 확인
df[["사업장명", "지번주소", "위도", "경도"]].head()

,사업장명,지번주소,위도,경도
0,고향집,서울특별시 종로구 숭인동 408-0번지,37.577555,127.019148
1,성현호프,서울특별시 종로구 명륜4가 154-3번지,37.582017,126.996896
2,방가방가,"서울특별시 종로구 관수동 20-0번지 21-7,21-8 국일관드림팰리스",37.569744,126.987088
3,비비알(B.B.R)비빔밥,"서울특별시 종로구 관수동 20-0번지 21-7,21-8 국일관드림팰리스",37.569744,126.987088
4,아침바다,서울특별시 종로구 내수동 110-33번지,37.572916,126.968694


## 📘 3. 마포구 음식점만 필터링

In [42]:
# 지번주소 기준 마포구만 필터링
df_mapo = df[df["지번주소"].str.contains("마포", na=False)]

# NaN 값 제거
df_mapo = df_mapo.dropna(subset=["위도", "경도"])

# 필요한 컬럼만 정리
df_mapo = df_mapo[["사업장명", "지번주소", "위도", "경도"]]
df_mapo.reset_index(drop=True, inplace=True)

# 저장 (선택)
df_mapo.to_csv("../data/processed/mapo_restaurants.csv", index=False)

# 확인
df_mapo.head()

,사업장명,지번주소,위도,경도
0,해적쭈꾸미,서울특별시 마포구 망원동 394-89번지 1층,37.552936,126.903311
1,보물섬,서울특별시 마포구 아현동 326-24번지,37.555927,126.951907
2,굴다리식당,"서울특별시 마포구 도화동 25-6번지 ,7",37.542255,126.948966
3,우리왕만두,서울특별시 마포구 대흥동 404-6번지,37.546144,126.938931
4,홀짝홀짝,서울특별시 마포구 서교동 355-25,37.553685,126.918553


In [44]:
df_mapo.shape

(26716, 4)